In [78]:
import pathlib
from typing import Dict, List, Union, Optional
from urllib.request import urlretrieve
import re
import requests
import os

import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point, Polygon

%matplotlib inline

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 150
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [66]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
DATA_DIR = pathlib.Path("data").resolve()
DATA_DIR

PosixPath('/home/matt/projects/analysis/us_employers/sec/data')

In [67]:
def extract_data_from_url(
    file_path: pathlib.Path,
    url: str,
    data_format: str,
    force_repull: bool = False,
) -> pd.DataFrame:
    file_path = pathlib.Path(file_path).resolve()
    file_path.parent.mkdir(exist_ok=True, parents=True)
    if not file_path.is_file() or force_repull:
        urlretrieve(url, file_path)
    if data_format in ["csv", "zipped_csv"]:
        return pd.read_csv(file_path)
    elif data_format in ["json"]:
        return pd.read_json(file_path)
    elif data_format in ["shp", "geojson"]:
        return gpd.read_file(file_path)

In [73]:
company_tickers_df = extract_data_from_url(
    file_path = DATA_DIR.joinpath("company_tickers.json"),
    url = "https://www.sec.gov/files/company_tickers.json",
    data_format="json",
)
company_tickers_df = company_tickers_df.T

In [74]:
print(company_ticker_df.shape)
company_ticker_df.head(2)

(12115, 3)


,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP


In [52]:
def decode_lines(response_lines):
    decoded_lines = []
    error_lines = []
    for line in response_lines:
        try:
            decoded_lines.append(line.decode("utf-8"))
        except Exception:
            error_lines.append(line)
            print(line)
    return (decoded_lines, error_lines)
        

In [76]:
url = "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt"

# urlretrieve(url=url, filename=out_path)

In [80]:
url = "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt"
resp = requests.get(
    url,
    headers={
        "User-Agent":os.environ["email"]
    }
)

In [81]:
resp.status_code

200

In [82]:
response_lines = [line for line in resp.iter_lines()]
decoded_lines, error_lines = decode_lines(response_lines)
company_cik_codes = pd.DataFrame(decoded_lines)
company_cik_codes_df = company_cik_codes[0].str.split(":", n=1, expand=True)
company_cik_codes_df = company_cik_codes_df.rename(columns={0:"company", 1:"cik"})
company_cik_codes_df["cik"] = company_cik_codes_df["cik"].str.replace(":", "")

b'EXPLORA INVESTIMENTOS GEST\xc3O DE RECURSOS LTDA:0001538467:'
b'PRAGMA GEST\xc3O DE PATRIM\xd4NIO LTDA:0001555512:'


In [83]:
print(company_cik_codes.shape)
company_cik_codes.head(2)

(881109, 1)


,0
0,!J INC:0001438823:
1,"#1 A LIFESAFER HOLDINGS, INC.:0001509607:"


In [90]:
company_cik_codes_df.to_csv(DATA_DIR.joinpath("edgar-cik-lookup-data.csv"), index=False)

In [89]:
company_cik_codes_df.head(2)

,company,cik
0,!J INC,0001438823
1,"#1 A LIFESAFER HOLDINGS, INC.",0001509607


In [36]:
response_lines = [line for line in resp.iter_lines()]

In [40]:
# response_lines[20:40]

In [ ]:
company_cik_codes

In [59]:
# [line.split(":", 1) for line in decoded_lines]

In [60]:
# decoded_lines

In [46]:
print(error_lines[0])

b'EXPLORA INVESTIMENTOS GEST\xc3O DE RECURSOS LTDA:0001538467:'
